## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
from pyspark.sql.functions import *

In [0]:
dbutils.widgets.text(name="storage_account", defaultValue= " ", label= "Storage Account String")
dbutils.widgets.text(name="container_name", defaultValue= " ", label= "container Name")
dbutils.widgets.text(name="mount_path",defaultValue=" ",label="Mount Path")
dbutils.widgets.text(name="storage_account_key", defaultValue= "Ends with ==", label= "Storage Account key")

In [0]:
# Create a spark dataframe in the 

In [0]:
# File location and type
file_location = "/FileStore/tables/computerlogs/CpuLogData2019*.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df = df.select("DateTime","keyboard","mouse","user_name")\
        .withColumn('Date', split(df['DateTime'], ' ').getItem(0))\
        .withColumn('Time', split(df['DateTime'], ' ').getItem(1))\
        .withColumn('DateTime', to_timestamp(df['DateTime']))
print(df.count())

In [0]:
storage_account_name =dbutils.widgets.get("storage_account")
# Azure Storage Account Key
storage_account_key =dbutils.widgets.get("storage_account_key")

# Azure Storage Account Source Container
container = dbutils.widgets.get("container_name")
mount_path=dbutils.widgets.get("mount_path")
# Set the configuration details to read/write
spark.conf.set("fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name), storage_account_key)

In [0]:
dbutils.fs.mount(
   source = "wasbs://{0}@{1}.blob.core.windows.net".format(container, storage_account_name),
   mount_point = mount_path,
   extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name): storage_account_key}
  )

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-3348328071542889> in <module> 
 2 source = "wasbs://{0}@{1}.blob.core.windows.net" . format ( container , storage_account_name ) , 
 3 mount_point = mount_path , 
 ----> 4 extra_configs = { "fs.azure.account.key.{0}.blob.core.windows.net" . format ( storage_account_name ) : storage_account_key } 
 5 )

 /local_disk0/tmp/1614313033070-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 322 exc . __context__ = None 
 323 exc . __cause__ = None 
 --> 324 raise exc
 325 return f_with_exception_handling
 326 

 ExecutionError : An error occurred while calling o339.mount.
: shaded.databricks.org.apache.hadoop.fs.azure.AzureException: java.lang.StringIndexOutOfBoundsException: String index out of range: 7
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.createAzureStorageSession(AzureNativeFileSystemStore.java:1037)
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.initialize(AzureNativeFileSystemStore.java:488)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.initialize(NativeAzureFileSystem.java:1352)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.verifyAzureFileSystem(DBUtilsCore.scala:502)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:451)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.StringIndexOutOfBoundsException: String index out of range: 7
	at java.lang.String.charAt(String.java:658)
	at hadoop_azure_shaded.com.microsoft.azure.storage.core.Base64.decode(Base64.java:84)
	at hadoop_azure_shaded.com.microsoft.azure.storage.StorageCredentialsAccountAndKey.<init>(StorageCredentialsAccountAndKey.java:81)
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.connectUsingConnectionStringCredentials(AzureNativeFileSystemStore.java:877)
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.createAzureStorageSession(AzureNativeFileSystemStore.java:1022)
	... 15 more

In [0]:
# Create a view or table

temp_table_name = "CpuLogData"

df.createOrReplaceTempView(temp_table_name)

In [0]:
highest_average_active_user_df = sqlContext.sql("select user_name,from_unixtime(ROUND(((count(*)*5)*60)/6,2),'HH:mm') as average_active_time_in_hours from `CpuLogData` where ( keyboard != 0 or mouse != 0 ) group by user_name order by average_active_time_in_hours desc")
display(highest_average_active_user_df)
highest_average_active_user_df.write.option("header",True) \
 .csv("/mnt/sparklogfile/highest_average_active_user_df.csv")

In [0]:
lowest_average_active_user_df = sqlContext.sql("select user_name,from_unixtime(ROUND(((count(*)*5)*60)/6,2),'HH:mm')  as average_active_time_in_hours from `CpuLogData` where (keyboard != 0 or mouse != 0 ) group by user_name order by average_active_time_in_hours limit 1")
display(lowest_average_active_user_df)
lowest_average_active_user_df.write.option("header",True) \
 .csv("/mnt/sparklogfile/lowest_average_active_user_df.csv")

In [0]:
# Highest Average Idle hour 
highest_average_idle_user_df = sqlContext.sql("select user_name,from_unixtime(ROUND(((count(*)*5)*60)/6,2),'HH:mm') as average_idle_time_in_hours from `CpuLogData` where (keyboard == 0 and mouse == 0 ) group by user_name order by average_idle_time_in_hours desc limit 1 ")
display(highest_average_idle_user_df)
highest_average_idle_user_df.write.option("header",True) \
 .csv("/mnt/sparklogfile/highest_average_idle_user_df.csv")

In [0]:

daily_session_start_time_df = sqlContext.sql("select Date,to_timestamp(min(DateTime)) as DateTime from CpuLogData group by Date order by Date")
temp_table_name = "view_session_start_time"
daily_session_start_time_df.createOrReplaceTempView(temp_table_name)
display(daily_session_start_time_df)


In [0]:
daily_login_time_df = sqlContext.sql("select user_name,Date,min(DateTime) as Datetime from `CpuLogData` where (keyboard != 0 or mouse != 0) group by user_name,Date order by Date")
temp_table_name = "view_daily_login_time"
daily_login_time_df.createOrReplaceTempView(temp_table_name)
display(daily_login_time_df)


In [0]:
late_users_df = sqlContext.sql("select lut.user_name,\
Concat(cast(Round((Sum(lut.TimeDifference)/60)/6,2) AS INT),' Hour , ',Floor(Round((((Sum(lut.TimeDifference)/60)/6)%1)*60)),' Min') as average_late_time_in_hours, count(*) as daily_late_count from (Select lt.user_name,(unix_timestamp(lt.DateTime)-unix_timestamp(st.DateTime))/(60) as TimeDifference from view_daily_login_time as lt,view_session_start_time as st where lt.Date == st.Date) as lut where lut.TimeDifference != 0 group by lut.user_name order by average_late_time_in_hours desc")
display(late_users_df)
late_users_df.write.option("header",True) \
 .csv("/mnt/sparklogfile/late_users_df.csv")